In [1]:
import sys, os
import pathlib
# include parent directory in Python path.
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import numpy as np
import pandas as pd
from utils import show_ram_usage
from models.linear import linear

In [2]:
train = pd.read_csv('../dataset/train.csv')
dipole_moments = pd.read_csv('../dataset/dipole_moments.csv')
magnetic_shielding_tensors = pd.read_csv('../dataset/magnetic_shielding_tensors.csv')
potential_energy = pd.read_csv('../dataset/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv('../dataset/scalar_coupling_contributions.csv')
structures = pd.read_csv('../dataset/structures.csv')

test = pd.read_csv('../dataset/test.csv')
submission = pd.read_csv('../dataset/sample_submission.csv')

In [3]:
print(train.shape)
train.head()

(4658147, 6)


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [4]:
print(structures.shape)
structures.head()

(2358657, 6)


,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [5]:
print(scalar_coupling_contributions.shape)
scalar_coupling_contributions.head()

(4658147, 8)


,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [6]:
print(magnetic_shielding_tensors.shape)
magnetic_shielding_tensors.head()

(1533537, 11)


,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [7]:
print(dipole_moments.shape)
dipole_moments.head()

(85003, 4)


,molecule_name,X,Y,Z
0,dsgdb9nsd_000001,0.0000,0.0,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937
4,dsgdb9nsd_000007,0.0000,0.0,0.0000


In [8]:
print(potential_energy.shape)
potential_energy.head(15)

(85003, 2)


,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.52368
1,dsgdb9nsd_000002,-56.56025
2,dsgdb9nsd_000003,-76.42608
3,dsgdb9nsd_000005,-93.42849
4,dsgdb9nsd_000007,-79.83869
5,dsgdb9nsd_000008,-115.73030
6,dsgdb9nsd_000009,-116.66500
7,dsgdb9nsd_000010,-132.76340
8,dsgdb9nsd_000011,-153.84300
9,dsgdb9nsd_000012,-169.90610


In [9]:
test.head(30)

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC
5,4658152,dsgdb9nsd_000015,3,0,1JHC
6,4658153,dsgdb9nsd_000015,3,2,3JHC
7,4658154,dsgdb9nsd_000015,3,4,2JHH
8,4658155,dsgdb9nsd_000015,3,5,2JHH
9,4658156,dsgdb9nsd_000015,4,0,1JHC


`potential_energy`'s `molecule_name` numbers does not seem to be included in `test`. Include!? Or How?

`scalar_coupling_contributions` has the same length as `train`, just merge?

In [10]:
assert train.shape[0] == scalar_coupling_contributions.shape[0]
train = pd.merge(train, scalar_coupling_contributions)
print(train.shape)
train.head()

(4658147, 10)


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,83.0224,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,-11.0347,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,-11.0325,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,-11.0319,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,83.0222,0.254585,1.25861,0.272013


In [11]:
# this is wrong. how to merge/join properly w/o doubling records!?!?!
#%time X = pd.merge(X, magnetic_shielding_tensors, on='molecule_name', how='inner')

In [12]:
y = train.scalar_coupling_constant
X = train.drop(['scalar_coupling_constant', 'molecule_name', 'id'], axis=1)

In [13]:
show_ram_usage()

RAM usage: 1.6670989990234375 GB


In [14]:
linear(X, y)

Training Score (R^2): 0.999999999994535
Best Parameters: {}
Mean absolute error: 2.222592408804712e-05


Pipeline(memory=None,
         steps=[('encoder',
                 OrdinalEncoder(cols=['type'], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[{'col': 'type',
                                          'data_type': dtype('O'),
                                          'mapping': 1JHC    1
2JHH    2
1JHN    3
2JHN    4
2JHC    5
3JHH    6
3JHC    7
3JHN    8
NaN    -2
dtype: int64}],
                                return_df=True, verbose=0)),
                ('lr',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [17]:
X.head()

,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,2,0,1JHC,83.0222,0.254585,1.25861,0.272013
